# Analysis Notebook - Hierarchical Bayesian Modelling

## **NOTE**:

We assume that you have cloned the analysis repository and have `cd` into the parent directory. Before starting with the analysis make sure you have first completed the dependencies set up by following the instructions described in the **`dependencies/README.md`** document. All paths defined in this Notebook are relative to the parent directory (repository). Please close this Notebook and start again by following the above guidelines if you have not completed the aforementioned steps.

## Prerequisite input files

Before starting the execution of the following code, make sure you have available in the folders `sbas/data` and `sbas/assets` the files listed below as prerequisites.

###  **`sbas/data`**.
The present analysis requires the following files to be present in the folder **`sbas/data`** uncompressed as the filenames indicate below:

- [x] `fromGTF.{as_site_type}.txt`
- [x] `GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct`
- [x] `rmats_final.{as_site_type}.jc.ijc.txt`
- [x] `rmats_final.{as_site_type}.jc.sjc.txt`
- [x] `limma::topTable()` dataframes written as csv, filenames shsould follow the convention: `{as_site_type}_{tissue_name}_AS_model_B_sex_as_events.csv`
- [x] `srr_pdata.csv`: the corrected GTEx data as created by the forked yarn and in the `annes-changes` branch https://github.com/TheJacksonLaboratory/yarn/tree/annes-changes with the SRR data as used in the `rMATS 3.2.5` experiment.

### **`sbas/assets`**
The present analysis requires the following files to be present in the folder **`sbas/assets`**.

- [x] `tissues.tsv`: metadata file with information on which tissues will be used for analysis -- only using the `include` option for analysis
- [x] `splice-relevant-genes.txt`: list of RNA binding proteins that are annotated to splicing relevant functions from GO.


**NOTE**: For convenience there are two `.tar.gz` archives with the contents described above.

```
gs://robinson-bucket/notebooks/bayesian-modeling/data_bayesian_se_AS_model_B_sex_as_events.tar.gz
gs://robinson-bucket/notebooks/bayesian-modeling/assets_bayesian_se_AS_model_B_sex_as_events.tar.gz
```

Before running the notebook, one can unpack the contents of the archives in the `sbas-nf/data` folder as required by executing the following commands:

```bash
# git clone https://github.com/TheJacksonLaboratory/sbas-nf
tar xvzf data_bayesian_{as_site_type}_AS_model_B_sex_as_events.tar.gz -C data/
tar xvzf assets_bayesian_{as_site_type}_AS_model_B_sex_as_events.tar.gz -C assets/

```


## Loading dependencies

If `conda` is available on your environment you can install the required dependencies by running the following commands:


```bash
time conda install -y r-base==3.6.2 &&
conda install -y r-ggplot2 r-ggsci r-coda r-rstan r-rjags r-compute.es r-snakecase &&
Rscript -e 'install.packages("runjags", repos = "https://cloud.r-project.org/")'
```



In [ ]:
# Start the clock!
start_time <- Sys.time()

In [ ]:
# dataviz dependencies
library(ggplot2)
library(ggsci)
library(grid)
library(gridExtra)
library(stringr)
library(snakecase)

# BDA2E-utilities dependencies
library(rstan)
library(parallel)
library(rjags)
library(runjags)
library(compute.es)

Previously used list of tissues to use for the Hierarchical Bayesian modelling:



```R
tissue.list<-c("Heart - Left Ventricle",
               "Breast - Mammary Tissue",
               "Brain - Cortex.Brain - Frontal Cortex (BA9).Brain - Anterior cingulate cortex (BA24)",
               "Adrenal Gland",
               "Adipose - Subcutaneous",
               "Muscle - Skeletal",
               "Thyroid",
               "Cells - Transformed fibroblasts",
               "Artery - Aorta",
               "Skin - Sun Exposed (Lower leg).Skin - Not Sun Exposed (Suprapubic)")
```

In [ ]:
tissues_df <- readr::read_delim("../assets/tissues.tsv", delim = "\t")

In [ ]:
tissue.list <- tissues_df$name[ tissues_df$include ==1]

In [ ]:
message(length(tissue.list), " tissues")
cat(tissue.list, sep = "\n")

In [ ]:
tissue      <- tissue.list[tissue_index]    #can be replaced with a loop or argument to choose a different tissue
as_site_type #can be replaced with a loop or argument to choose a different as_site_type

In [ ]:
tissue
as_site_type

## Pattern for choosing `topTable()` files from `limma`

```bash
# {as_site_type} + '_' + {tissue} + '_' + suffix_pattern 
se_skin_not_sun_exposed_suprapubic_AS_model_B_sex_as_events.csv
```

In [ ]:
tissue
dataDir <- "../data/"
assetsDir <- "../assets/"
as_site_type
suffix_pattern <- "AS_model_B_sex_as_events.csv"

file.with.de.results <- paste0(dataDir, as_site_type, "_", tissue, "_" , suffix_pattern  )
file.with.de.results
file.exists(file.with.de.results)
system( paste0("ls -l ", file.with.de.results), intern = TRUE )

**NOTE**:
The parameters `tissue_index` and `as_site_type` will be passed from papermill and added in the fist chunk of the notebook upon execution. Do not assign to a hardcoded value anywhere in the notebook, as they will ovewrite the assignment in the first chunk.

In [ ]:
events.table         <- read.table(file.with.de.results, sep = ",")
head(events.table, 2)

## Add annotation columns to the topTable dataframe:

The feature information is encoded in the topTable dataframe as rownames. The `ID` and `geneSymbol` variables have been combined in the following pattern:

```console
{geneSymbol}-{ID} 
```

- `ID`: everything **_after_** last occurence of hyphen `-`
example: 
```R
stringr::str_replace("apples - oranges - bananas", "^.+-", "")
```

```console
# output:

' bananas'
```

- `geneSymbol`: everything **_before_** last occurence of `-`
example: 

```R
sub('-[^-]*$', '',"apples - oranges - bananas")
```

```console
# output:

'apples - oranges '
```

```diff
- NOTE: The above solution covers the cases where a hyphen is part of the geneSymbol.
```

In [ ]:
cols_initially <- colnames(events.table)
cols_initially

In [ ]:
events.table[["ID"]] <- stringr::str_replace(rownames(events.table),  "^.+-", "")
events.table[["gene_name"]] <- sub('-[^-]*$', '', rownames(events.table))

In [ ]:
keepInOrderCols <- c("gene_name", "ID", cols_initially)

In [ ]:
events.table <- events.table[ , keepInOrderCols ]

In [ ]:
tail(events.table, 2)

## Define filepaths of required inputs

`file.with.de.results` has been defined above

In [ ]:
rbp.table.name        <- paste0(assetsDir, "splice-relevant-genes.txt")
file.exists(rbp.table.name)

In [ ]:
events.table.name     <- paste0(paste0(paste0(dataDir, "fromGTF."), snakecase::to_upper_lower_case(as_site_type)),".txt")
events.table.name
file.exists(events.table.name)

In [ ]:
inc.counts.file.name  <- paste0(paste0(paste0(dataDir, "rmats_final."), as_site_type),".jc.ijc.txt")
inc.counts.file.name
file.exists(inc.counts.file.name)

In [ ]:
skip.counts.file.name  <- paste0(paste0(paste0(dataDir, "rmats_final."), as_site_type),".jc.sjc.txt")
skip.counts.file.name
file.exists(skip.counts.file.name)

In [ ]:
metadata.file.name    <- paste0(dataDir, "srr_pdata.csv")
file.exists(metadata.file.name)

In [ ]:
expression.file.name  <- paste0(dataDir, "GTEx_Analysis_2017-06-05_v8_RNASeQCv1.1.9_gene_tpm.gct")
file.exists(expression.file.name)

## Use the define filepaths to load/read in the tables 

Load the sjc and sjc count matrices, and the list of RNA binding proteins that are annotated to either:
- mRNA splicing, via spliceosome `(GO:0000398)`,
- regulation of mRNA splicing, via spliceosome `(GO:0048024)`, or 
- both. 

The table has the:
- Gene Symbol
- the Uniprot ID (`uprot.id`)
- the NCBI Gene ID (`gene.id`) and 
- boolean columns for being 
  - `S`=mRNA splicing, via spliceosome `(GO:0000398)` and 
  - `R`=regulation of mRNA splicing, via spliceosome `(GO:0048024)`.

### Filtering of the `topTable()` object

- `abs(events.table$logFC)>=log2(1.5)`
- `events.table$adj.P.Val<=0.05`

In [ ]:
dim(events.table)
events.table <- events.table[abs(events.table$logFC)>=log2(1.5) & events.table$adj.P.Val<=0.05,]
dim(events.table)
head(events.table,2)

Make sure this command has been executed before `gunzip sbas/data/fromGTF.*` as the files are expected uncompressed.


In [ ]:
annot.table  <- read.table(events.table.name,header=T)
dim(annot.table)
head(annot.table, 1)

In [ ]:
merged.table <- merge(events.table, annot.table, by="ID")

In [ ]:
dim(merged.table)
head(merged.table, 2)

In [ ]:
rbp.table    <- read.table(rbp.table.name,sep="\t",header=TRUE)
dim(rbp.table)
head(rbp.table, 1)

Make sure this command has been executed before `gunzip sbas/data/rmats_final.{as_site_type}.jc.*jc.*` as the files are expected uncompressed.


In [ ]:
inc.counts   <- as.data.frame(data.table::fread(inc.counts.file.name))
dim(inc.counts)
inc.counts[1:2,1:3]

In [ ]:
skip.counts  <- as.data.frame(data.table::fread(skip.counts.file.name))
dim(skip.counts)
skip.counts[1:2,1:3]

## Check `dim()` of loaded objects

In [ ]:
dim(events.table)
dim(annot.table)
dim(merged.table)
dim(rbp.table)
dim(inc.counts)
dim(skip.counts)

## Read sample info

In [ ]:
metadata.file.name
file.exists(metadata.file.name)
system(paste0("ls -l", " ../data/srr_pdata.csv"), intern = TRUE)

In [ ]:
meta.data <- readr::read_csv(metadata.file.name)
dim(meta.data)
head(meta.data, 1)

In [ ]:
meta.data$SMTSD[1:3]

In [ ]:
meta.data[["SMTSD"]] <- as.character(meta.data[["SMTSD"]])

In [ ]:
meta.data$SMTSD[1:3]

In [ ]:
meta.data <- meta.data[ snakecase::to_snake_case(meta.data$SMTSD) == tissue,]

In [ ]:
tissue
dim(meta.data)
meta.data[1:2,1:8]

In [ ]:
# Undo snakecase of SMTSD
tissue
tissue <- unique(meta.data$SMTSD)
tissue

In [ ]:
dim(inc.counts)
inc.counts   <- inc.counts[,colnames(inc.counts) %in% meta.data$SRR]
dim(inc.counts)

In [ ]:
dim(skip.counts)
skip.counts  <- skip.counts[,colnames(skip.counts) %in% meta.data$SRR]
dim(skip.counts)

## This notebook only signficant events are used - filtering now on Adj.P Val

If there are more than 100 events that are significant, reduce this by ordering on the adjusted p-value (column Adj.P.Val).  

Then update the ijc and sjc matrices and the merged.table (annotations)


In [ ]:
if (nrow(skip.counts) > 100 ) {
    head(merged.table,2)
    dim(merged.table)
    order_by_adj.P.Val <- order(merged.table$adj.P.Val,decreasing=TRUE)
    head(merged.table[order_by_adj.P.Val,],2)
    keep_IDs <- merged.table[order_by_adj.P.Val,]$ID[1:100]
    head(keep_IDs)
    keep_IDs <- keep_IDs[order(keep_IDs, decreasing=FALSE)]
    head(keep_IDs)
    # counts table will be identical in rows
    counts.keep <- as.character(rownames(skip.counts)) %in% as.character(keep_IDs)
    table(counts.keep)
    # not so sure about merged.table - so lets have a different logical key
    merged.keep <- as.character(merged.table$ID) %in% as.character(keep_IDs)
    table(merged.keep)
    
    # update the sjc counts 
    dim(skip.counts)
    skip.counts  <- skip.counts[counts.keep,]
    dim(skip.counts)
 
    # update the ijc counts
    dim(inc.counts)
    inc.counts   <- inc.counts[counts.keep,]
    dim(inc.counts)
    
    # merged table
    dim(merged.table)
    merged.table       <- merged.table[merged.keep,] 
    dim(merged.table)
}

In [ ]:
dim(inc.counts)
dim(skip.counts)
dim(merged.table)

## Save R objects

In [ ]:
save.image(file = "notebook.RData")
file.exists("notebook.RData")
system("pwd && ls -l notebook.RData", intern = TRUE)

## Metadata

For replicability and reproducibility purposes, we also print the following metadata:

1. Checksums of **"artefacts"**, files generated during the analysis and stored in the folder directory **`data`**
2. List of environment metadata, dependencies, versions of libraries using `utils::sessionInfo()` and [`devtools::session_info()`](https://devtools.r-lib.org/reference/session_info.html)

### 1. Checksums with the sha256 algorithm

In [ ]:
figure_id       <- "bayesian-modeling"

message("Generating sha256 checksums of the artefacts in the `..data/` directory .. ")
system(paste0("cd ../data/ && find . -type f -exec sha256sum {} \\; > ../metadata/",  figure_id, "_sha256sums.txt"), intern = TRUE)
message("Done!\n")

data.table::fread(paste0("../metadata/", figure_id, "_sha256sums.txt"), header = FALSE, col.names = c("sha256sum", "file"))

### 2. Libraries metadata

In [ ]:
dev_session_info   <- devtools::session_info()
utils_session_info <- utils::sessionInfo()

message("Saving `devtools::session_info()` objects in ../metadata/devtools_session_info.rds  ..")
saveRDS(dev_session_info, file = paste0("../metadata/", figure_id, "_devtools_session_info.rds"))
message("Done!\n")

message("Saving `utils::sessionInfo()` objects in ../metadata/utils_session_info.rds  ..")
saveRDS(utils_session_info, file = paste0("../metadata/", figure_id ,"_utils_info.rds"))
message("Done!\n")

dev_session_info$platform
dev_session_info$packages[dev_session_info$packages$attached==TRUE, ]